In [ ]:
import pyodbc
import pandas as pd

# Conexão
server = "ekp-svr-datafinance.database.windows.net"
database = "BD_Dados_Finance"
username = "admin-db"
password = "Pedrica1009@"
driver = '{ODBC Driver 18 for SQL Server}'  # certifique-se que esse driver está instalado

# Conectar
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
)

cursor = conn.cursor()

# Criação da tabela, se não existir
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='NVDA' AND xtype='U')
BEGIN
    CREATE TABLE NVDA (
        Id INT PRIMARY KEY IDENTITY(1,1),
        ticket NVARCHAR(10),
        year INT NOT NULL,
        month INT NOT NULL,
        day INT NOT NULL,
        time TIME,
        [open] FLOAT NOT NULL,
        [high] FLOAT NOT NULL,
        [low] FLOAT NOT NULL,
        [close] FLOAT NOT NULL,
        [volume] FLOAT NOT NULL
    )
END
""")
conn.commit()

# Carregar CSV
dados = pd.read_csv("03 - Gold\\dados_load.csv")

# Preparar lista de tuplas
valores = [
    (
        "NVDA", 
        int(row['year']),
        int(row['month']),
        int(row['day']),
        row['time'],
        float(row['open']),
        float(row['high']),
        float(row['low']),
        float(row['close']),
        float(row['volume'])
    )
    for _, row in dados.iterrows()
]

# Ativar fast executemany
cursor.fast_executemany = True

# Inserir em lote
cursor.executemany("""
    INSERT INTO NVDA (ticket, year, month, day, time, [open], [high], [low], [close], [volume])
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", valores)

conn.commit()

# Testar leitura
cursor.execute("SELECT TOP 5 * FROM NVDA")
for row in cursor.fetchall():
    print(row)

# Fechar conexão
cursor.close()
conn.close()


(5693, 'NVDA', 2021, 1, 29, datetime.time(19, 55), 12.9095, 12.9095, 12.9095, 12.9095, 4600.0)
(5694, 'NVDA', 2021, 1, 29, datetime.time(19, 50), 12.9219, 12.9307, 12.9219, 12.9307, 30760.0)
(5695, 'NVDA', 2021, 1, 29, datetime.time(19, 45), 12.9282, 12.9282, 12.9282, 12.9282, 4000.0)
(5696, 'NVDA', 2021, 1, 29, datetime.time(19, 40), 12.9282, 12.9282, 12.9282, 12.9282, 16040.0)
(5697, 'NVDA', 2021, 1, 29, datetime.time(19, 35), 12.9157, 12.9157, 12.9157, 12.9157, 8000.0)
